<a href="https://colab.research.google.com/github/base-datos-empresas/Email-verifier/blob/main/Verificador_de_emails_de_bases_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar dependencias
!pip install dnspython tqdm pyisemail

import os
import pandas as pd
import dns.resolver
from google.colab import files
from tqdm import tqdm
import pyisemail
import threading

# Instrucciones de uso en consola
def mostrar_instrucciones():
    instrucciones = """
    Instrucciones de uso:
    1. Suba un archivo Excel que contenga una columna llamada 'email' con las direcciones de correo electrónico a verificar.
    2. Elija el modo de verificación:
       - '1' para Modo Normal: Verificaciones básicas (formato y dominio).
       - '2' para Modo Avanzado: Verificaciones intermedias (formato, dominio y registros MX).
       - '3' para Modo Ultra Avanzado: Verificaciones avanzadas (incluye SPF, DMARC, DKIM, dominio desechable y servidor SMTP).
       - Si no selecciona ningún modo en 20 segundos, se seleccionará el Modo Normal por defecto.
    3. Obtendrá un archivo CSV de salida con los emails válidos y sus estados detallados.

    Nota: Este código ha sido proporcionado por centraldecomunicacion.es.
    Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/
    """
    print(instrucciones)

# Funciones de verificación
def verificar_formato_email(email):
    """
    Verifica que el formato del email sea correcto utilizando pyisemail.
    """
    return pyisemail.is_email(email, check_dns=False)

def verificar_dominio(email):
    """
    Verifica que el dominio del email tenga registros DNS válidos.
    """
    dominio = email.split('@')[-1]
    try:
        dns.resolver.resolve(dominio, 'A')
        return True
    except dns.exception.DNSException:
        return False

def verificar_MX(email):
    """
    Verifica que el dominio del email tenga registros MX válidos.
    """
    dominio = email.split('@')[-1]
    try:
        registros_mx = dns.resolver.resolve(dominio, 'MX')
        return len(registros_mx) > 0
    except dns.exception.DNSException:
        return False

def verificar_registros_SPF(dominio):
    """
    Verifica si el dominio tiene registros SPF válidos.
    """
    try:
        registros_spf = dns.resolver.resolve(dominio, 'TXT')
        for txt_record in registros_spf:
            if 'v=spf1' in str(txt_record).lower():
                return True
        return False
    except dns.exception.DNSException:
        return False

def verificar_registros_DMARC(dominio):
    """
    Verifica si el dominio tiene una política DMARC.
    """
    try:
        registros_dmarc = dns.resolver.resolve('_dmarc.' + dominio, 'TXT')
        for txt_record in registros_dmarc:
            if 'v=dmarc1' in str(txt_record).lower():
                return True
        return False
    except dns.exception.DNSException:
        return False

def verificar_registros_DKIM(dominio):
    """
    Verifica si el dominio tiene registros DKIM.
    """
    try:
        # Lista de selectores comunes
        selectores = ['default', 'dkim', 'selector1', 'selector2', 'mail']
        for selector in selectores:
            registros_dkim = dns.resolver.resolve(f'{selector}._domainkey.{dominio}', 'TXT')
            for txt_record in registros_dkim:
                if 'v=dkim1' in str(txt_record).lower():
                    return True
        return False
    except dns.exception.DNSException:
        return False

def verificar_servidor_SMTP(email):
    """
    Verifica si el servidor SMTP del dominio está activo (prueba de conexión al puerto 25).
    """
    import smtplib
    dominio = email.split('@')[-1]
    try:
        registros_mx = dns.resolver.resolve(dominio, 'MX')
        mx_record = str(min(registros_mx, key=lambda r: r.preference).exchange)
        server = smtplib.SMTP(timeout=5)
        server.connect(mx_record)
        server.quit()
        return True
    except Exception:
        return False

def verificar_disposable_email(email):
    """
    Verifica si el email es de un dominio desechable conocido.
    """
    disposable_domains = set()
    # Lista de dominios desechables comunes (puedes ampliar esta lista)
    disposable_domains.update(['mailinator.com', 'trashmail.com', 'tempmail.com', '10minutemail.com'])
    dominio = email.split('@')[-1]
    return dominio in disposable_domains

def verificar_existencia_email(email, modo='normal'):
    """
    Verifica la existencia del correo electrónico según el modo seleccionado.
    Retorna un mensaje de estado.
    """
    if not verificar_formato_email(email):
        return 'Formato inválido'
    if not verificar_dominio(email):
        return 'Dominio inválido'

    if modo == 'normal':
        return 'Válido'

    if not verificar_MX(email):
        return 'Sin registros MX'

    if modo == 'avanzado':
        return 'Válido'

    dominio = email.split('@')[-1]
    if not verificar_registros_SPF(dominio):
        return 'Sin registros SPF'
    if not verificar_registros_DMARC(dominio):
        return 'Sin registros DMARC'
    if not verificar_registros_DKIM(dominio):
        return 'Sin registros DKIM'
    if verificar_disposable_email(email):
        return 'Dominio desechable'
    if not verificar_servidor_SMTP(email):
        return 'Servidor SMTP no responde'

    return 'Válido'

# Función para verificar y etiquetar emails en un DataFrame
def verificar_y_etiquetar_emails(df, modo='normal'):
    """
    Verifica y etiqueta los correos electrónicos en el DataFrame.
    """
    if 'email' not in df.columns:
        print("La columna 'email' no existe en este DataFrame.")
        return df, {}

    estadisticas = {}
    df['Estado'] = ''
    emails_a_verificar = df['email']

    print(f"\nIniciando verificación en Modo '{modo.upper()}'.")
    if modo == 'normal':
        print("Verificaciones: Formato y Dominio.")
    elif modo == 'avanzado':
        print("Verificaciones: Formato, Dominio y registros MX.")
    elif modo == 'ultra-avanzado':
        print("Verificaciones: Formato, Dominio, MX, SPF, DMARC, DKIM, dominio desechable y servidor SMTP.")

    for idx, email in emails_a_verificar.items():
        if not pd.isna(email):
            estado = verificar_existencia_email(email, modo=modo)
            df.at[idx, 'Estado'] = estado
            estadisticas[estado] = estadisticas.get(estado, 0) + 1
            print(f"Email: {email} | Estado: {estado}")
        else:
            df.at[idx, 'Estado'] = 'Email vacío'
            estadisticas['Email vacío'] = estadisticas.get('Email vacío', 0) + 1
            print(f"Email vacío en la fila {idx}")

    return df, estadisticas

# Función principal para procesar el archivo subido
def procesar_archivo(archivo, modo_verificacion):
    """
    Procesa el archivo subido, verifica y etiqueta los correos electrónicos.
    Genera un archivo CSV con los emails válidos y sus estados.
    """
    nombre_archivo_base, _ = os.path.splitext(archivo)
    ruta_archivo_final = nombre_archivo_base + f'_{modo_verificacion.upper()}.csv'

    with pd.ExcelFile(archivo, engine='openpyxl') as xls:
        df = pd.read_excel(xls, sheet_name=0)  # Solo la primera hoja
        print(f"\nProcesando hoja: {xls.sheet_names[0]} (verificación {modo_verificacion})")
        df_modificado, estadisticas = verificar_y_etiquetar_emails(df.copy(), modo=modo_verificacion)

        # Guardar el DataFrame en un archivo CSV
        df_modificado.to_csv(ruta_archivo_final, index=False)

    # Descargar archivo
    print("\nDescargando archivo...")
    files.download(ruta_archivo_final)
    print("Descarga completada.")

    return estadisticas

# Función para obtener la entrada del usuario con timeout
def input_con_timeout(prompt, timeout):
    import sys
    import time
    import threading

    def inner():
        try:
            return input(prompt)
        except EOFError:
            return ''

    user_input = [None]

    def get_input():
        user_input[0] = inner()

    thread = threading.Thread(target=get_input)
    thread.start()
    thread.join(timeout)
    if thread.is_alive():
        print("\nNo se ha seleccionado ningún modo. Continuando con el Modo Normal por defecto.")
        return ''
    else:
        return user_input[0]

# Mostrar instrucciones
mostrar_instrucciones()

# Cargar archivo
uploaded = files.upload()
for filename in uploaded.keys():
    print(f'\nArchivo subido: {filename}')
    modo = input_con_timeout("Elige el modo (escribe '1' para Modo Normal, '2' para Modo Avanzado, '3' para Modo Ultra Avanzado): ", 20).strip()
    if modo == '1' or modo == '':
        modo_verificacion = 'normal'
    elif modo == '2':
        modo_verificacion = 'avanzado'
    elif modo == '3':
        modo_verificacion = 'ultra-avanzado'
    else:
        print("Modo inválido seleccionado. Usando Modo Normal por defecto.")
        modo_verificacion = 'normal'

    estadisticas = procesar_archivo(filename, modo_verificacion)

    print("\n=== Estadísticas de verificación ===")
    total_verificados = sum(estadisticas.values())
    for estado, cantidad in estadisticas.items():
        print(f"{estado}: {cantidad}")
    print(f"Total de correos electrónicos verificados: {total_verificados}")

print("\nEste código ha sido proporcionado por centraldecomunicacion.es.")
print("Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/")



    Instrucciones de uso:
    1. Suba un archivo Excel que contenga una columna llamada 'email' con las direcciones de correo electrónico a verificar.
    2. Elija el modo de verificación:
       - '1' para Modo Normal: Verificaciones básicas (formato y dominio).
       - '2' para Modo Avanzado: Verificaciones intermedias (formato, dominio y registros MX).
       - '3' para Modo Ultra Avanzado: Verificaciones avanzadas (incluye SPF, DMARC, DKIM, dominio desechable y servidor SMTP).
       - Si no selecciona ningún modo en 20 segundos, se seleccionará el Modo Normal por defecto.
    3. Obtendrá un archivo CSV de salida con los emails válidos y sus estados detallados.

    Nota: Este código ha sido proporcionado por centraldecomunicacion.es.
    Puede obtener bases de datos de empresas de forma gratuita y premium en: https://www.centraldecomunicacion.es/
    
